<a href="https://colab.research.google.com/github/SavageGinny/MLP-Jupiters/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pymorphy2 pymorphy3 natasha

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 94.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=3ceee908c3ccd552c3197fe860f342d4a56ce13acd37a4e0724a3125dab1f988
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=45eb9e49812ec67cc875b5d8d04475fc188a698d2c443744975d4af70e784aec
  Stored in directory: 

Загружаем библиотеки

In [5]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import pymorphy2
import pymorphy3
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger
import numpy as np


# Загрузка необходимых ресурсов nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Загрузка необходимых ресурсов nltk


In [6]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Текст

In [7]:
sample_text = "Пример обработки текста в Python :)"

Лемантизация и стеминг на разных библиотеках

In [8]:
def lemmatize_and_stem(text):
    morph_p2 = pymorphy2.MorphAnalyzer()
    morph_p3 = pymorphy3.MorphAnalyzer()
    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    morph_vocab = MorphVocab()

    stemmer = SnowballStemmer("russian")

    words = word_tokenize(text, language='russian')

    results = {
        "original": words,
        "pymorphy2": [morph_p2.parse(word)[0].normal_form for word in words],
        "pymorphy3": [morph_p3.parse(word)[0].normal_form for word in words],
        "natasha": []
    }

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        results["natasha"].append(token.lemma)

    results["stemmed"] = [stemmer.stem(word) for word in words]

    return results

lemmatized_stemmed = lemmatize_and_stem(sample_text)

Смотрим, что получилось

In [9]:
lemmatized_stemmed

{'original': ['Пример', 'обработки', 'текста', 'в', 'Python', ':', ')'],
 'pymorphy2': ['пример', 'обработка', 'текст', 'в', 'python', ':', ')'],
 'pymorphy3': ['пример', 'обработка', 'текст', 'в', 'python', ':', ')'],
 'natasha': ['пример', 'обработка', 'текст', 'в', 'python', ':)'],
 'stemmed': ['пример', 'обработк', 'текст', 'в', 'Python', ':', ')']}

Токенизатор

In [10]:
def tokenize_ascii_rus(text):
    return [char for char in text if ord(char)>1000]

In [11]:
tokenize_ascii_rus(" ".join(lemmatized_stemmed["original"]))

['П',
 'р',
 'и',
 'м',
 'е',
 'р',
 'о',
 'б',
 'р',
 'а',
 'б',
 'о',
 'т',
 'к',
 'и',
 'т',
 'е',
 'к',
 'с',
 'т',
 'а',
 'в']

Векторизатор

In [12]:
def vectorize_ascii_rus(text):
    return np.array([ord(char) for char in text if ord(char)>1000])

In [14]:
vectorize_ascii_rus(" ".join(lemmatized_stemmed["original"]))

array([1055, 1088, 1080, 1084, 1077, 1088, 1086, 1073, 1088, 1072, 1073,
       1086, 1090, 1082, 1080, 1090, 1077, 1082, 1089, 1090, 1072, 1074])